# Install pipelines SDK

#### Please wait till this cell completes and then run next cells. This just need to be run once per active kernel.

In [1]:
%env RELEASE_VERSION=1.0.0
!pip install https://storage.googleapis.com/ml-pipeline/release/${RELEASE_VERSION}/kfp.tar.gz --upgrade

env: RELEASE_VERSION=1.0.0
  Using cached https://storage.googleapis.com/ml-pipeline/release/1.0.0/kfp.tar.gz (122 kB)
  Created wheel for kfp: filename=kfp-1.0.0-py3-none-any.whl size=167371 sha256=0db4c79a58b100ae045ac851efa75f24f50988549675c32bfdb96e0d906818bb
  Stored in directory: /tmp/pip-ephem-wheel-cache-5zjv0esl/wheels/46/ec/3d/7ce52c89bbbc341040f126eaca22d6a02071f01137e5cd5757
Successfully built kfp
  Attempting uninstall: kfp
    Found existing installation: kfp 1.0.0
    Uninstalling kfp-1.0.0:
      Successfully uninstalled kfp-1.0.0


# Import kfp pkgs

In [2]:
import kfp
import kfp.dsl as dsl
import kfp.compiler as compiler
from kubernetes import client as k8s_client

# List existing pipeline experiments

In [3]:
client = kfp.Client()
client.list_experiments()

{'experiments': [{'created_at': datetime.datetime(2020, 8, 25, 5, 28, 10, tzinfo=tzlocal()),
                  'description': 'All runs created without specifying an '
                                 'experiment will be grouped here.',
                  'id': '117479f1-d8d8-4f9a-82d0-9cedae220e55',
                  'name': 'Default',
                  'resource_references': None,
                  'storage_state': None},
                 {'created_at': datetime.datetime(2020, 8, 27, 18, 36, 10, tzinfo=tzlocal()),
                  'description': None,
                  'id': '6a0bd354-71c8-4644-a94e-d43c87a5e7b4',
                  'name': 'Dkube - Mnist pl',
                  'resource_references': None,
                  'storage_state': None}],
 'next_page_token': None,
 'total_size': 2}

# Create a Dkube RESNETv2 experiment

In [4]:
resnet_experiment = client.create_experiment(name='Dkube - Resnetv2 pl')

# Define e2e RESENTv2 Pipeline with Dkube components

In [5]:
import kfp.dsl as dsl
from kfp import components
from kubernetes import client as k8s_client

import os
import json
from random import randint

dkube_training_op           = components.load_component_from_file("../components/training/component.yaml")
dkube_serving_op            = components.load_component_from_file("../components/serving/component.yaml")
dkube_viewer_op             = components.load_component_from_file('../components/viewer/component.yaml')

@dsl.pipeline(
    name='resnetv2',
    description='sample resnetv2 model based catsdogs classification pipeline with dkube components'
)
def d3pipeline(
    #Name of the project in dkube
    training_program,
    #Dataset to train on
    training_dataset,
    #Output model 
    training_output_model,
    #Dkube authentication token
    auth_token= os.getenv("DKUBE_USER_ACCESS_TOKEN"),
    #By default 'default' is used as the job group for runs
    job_group = 'default',
    #Framework. One of tensorflow, pytorch, sklearn
    framework = "tensorflow",
    #Framework version
    version = '1.14',
    #By default tf v1.14 image is used here, v1.13 or v1.14 can be used. 
    #Or any other custom image name can be supplied.
    #For custom private images, please input username/password
    training_container=json.dumps({'image':'docker.io/ocdr/d3-datascience-tf-cpu:v1.14', 'username':'', 'password': ''}),
    #Script to run inside the training container
    training_script="python model.py",
    #Input dataset mount paths
    training_input_dataset_mount="/opt/dkube/input",
    #Output models for training.
    training_output_mount="/opt/dkube/output",
    #Request gpus as needed. Val 0 means no gpu, then training_container=docker.io/ocdr/dkube-datascience-tf-cpu:v1.12.
    training_gpus=0,
    #Any envs to be passed to the training program
    training_envs=json.dumps([{"steps": 100}]),
    #Serving image
    serving_image=json.dumps({'image':'docker.io/ocdr/tensorflowserver:1.14', 'username':'', 'password': ''}),
    #Transformer image
    transformer_image=json.dumps({'image':'docker.io/ocdr/catdogs-example-preprocess:2.0.7', 'username':'', 'password': ''}),
    #Name of the transformer project in dkube. Update accordingly if different name is used while creating a workspace in dkube.
    transformer_project="catsdogs-transformer",
    #Script to execute the transformer
    transformer_code="tensorflow/classification/resnet/catsdogs/transformer/transformer.py"
):

    train       = dkube_training_op(auth_token, training_container,
                                    program=training_program, run_script=training_script,
                                    datasets=json.dumps([str(training_dataset)]), outputs=json.dumps([str(training_output_model)]),
                                    input_dataset_mounts=json.dumps([str(training_input_dataset_mount)]),
                                    output_mounts=json.dumps([str(training_output_mount)]),
                                    ngpus=training_gpus,
                                    envs=training_envs, job_group=job_group,
                                    framework=framework, version=version)
    serving     = dkube_serving_op(auth_token, train.outputs['artifact'],
                                serving_image=serving_image,
                                transformer_image=transformer_image,
                                transformer_project=transformer_project,
                                transformer_code=transformer_code).after(train)

# Compile and generate tar ball

In [7]:
compiler.Compiler().compile(d3pipeline, 'dkube_resnetv2_pl.tar.gz')
# Upload this generated tarball into the Pipelines UI

# Create and Run pipeline

#### Click the dkube-training stage to see the enhanced Dkube Datascience dashboard, metrics and graphs. Click the dkube-viewer stage for the simple UI to test the model predecitions

In [8]:
run = client.run_pipeline(resnet_experiment.id, 'resnetv2_classifier_pipeline', 'dkube_resnetv2_pl.tar.gz', params={})